In [1]:
remotes::install_github("ropensci/rmangal")

Skipping install of 'rmangal' from a github remote, the SHA1 (87df6fd8) has not changed since last install.
  Use `force = TRUE` to force installation



In [2]:
library(tidyverse)
library(magrittr)
library(dplyr)
library(sf)
library(rmangal)
library(raster)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract


Linking to GEOS 3.10.2, GDAL 3.5.0, PROJ 9.0.0; sf_use_s2() is TRUE

Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:dplyr’:

    select




In [52]:
networks_metadata_path = "../../data/networks/all/networks_metadata.csv"

rmangal_observations_data_path = "../../data/networks/metadata/parsed_rmangal_networks_metadata.csv"
rmangal_metadata_path = "../../data/networks/metadata/all_rmangal_networks.csv"

ecoregion_polygon_data_path1 = "../../data/networks/metadata/polygon/WWF/wwf_terr_ecos.shp"
ecoregion_polygon_data_path2 = "../../data/networks/metadata/polygon/Ecoregions2017/Ecoregions2017.shp"

temp_input_for_bioclim_path = "../../data/networks/all/geo_avail_networks_metadata.csv"
temp_output_for_bioclim_path = "../../data/networks/all/geo_extracted_networks_metadata.csv"
bioclim_dir = "../../data/networks/metadata/bioclimate/"
richness_data_path = "../../data/networks/metadata/Richness_2021.tif"
rmangal_geodata_path = "../../data/networks/rmangal_networks_geodata.csv"

# Mangal

In [11]:
if (!file.exists(rmangal_metadata_path))
{
    pollination_interactions<- search_interactions(type = "mutualism")
    network_ids = unique(pollination_interactions$network_id)
    length(network_ids)
    
    all_networks_data = data.frame()
    network_metadata_dfs = list()
    num_avail = 0
    for (i in 1:length(network_ids)) #length(pollination_networks)
    {
      if (i %% 10 == 0) 
      {
        print(i)
        print(num_avail)
      }
      networks = search_networks(list(id=nth(network_ids,i)))  %>% get_collection
      network = networks[[1]]
      network_data = network$network[c("geom_lat", "geom_type", "geom_lon")]
      network_ref_data = network$reference
      network_data = bind_cols(network_data, network_ref_data)
      network_data[["network_id"]] = nth(network_ids,i)
      network_metadata_dfs[[i]] = network_data
      if (! is.na(network_data$geom_lon))
      {
        num_avail = num_avail + 1
      }
    df = ldply(network_metadata_dfs, rbind)
    df <- apply(df,2,as.character)
    write.csv(df,rmangal_metadata_path)
    }
}

In [5]:
rmangal_metadata = read.csv(rmangal_metadata_path)

In [8]:
head(rmangal_metadata[is.na(rmangal_metadata$geom_lat), ][c("network_id", "doi", "paper_url", "geom_lon", "geom_lat")])

,network_id,doi,paper_url,geom_lon,geom_lat
,<int>,<chr>,<chr>,<chr>,<chr>
6,1419,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA
9,952,NA,null,NA,NA
10,1420,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA
19,1310,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA
20,1311,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA
21,1307,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA


In [ ]:
# merge rmangal_metadata to networks_metadata

# Web of life

In [88]:
base_url <- "https://www.web-of-life.es/"
networks_metadata = read.csv(networks_metadata_path)
non_wol_networks_metadata = networks_metadata[networks_metadata$source != "web_of_life", ] 
wol_networks_metadata = networks_metadata[networks_metadata$source == "web_of_life", ][c('network_type','network_index','path','source','is_legal','is_resolved','processed_path','is_parsed_legal')]                                                                                      
all_nw_info <- read.csv(paste0(base_url,"get_network_info.php"))

In [89]:
get_name <- function(path)
{
    path_name = basename(path)
    name = unlist(strsplit(path_name, ".", fixed=TRUE))[1]
    return (name)
}

In [90]:
wol_networks_metadata$network_name = unlist(lapply(wol_networks_metadata$path,get_name))

In [91]:
all_nw_info$Latitude <- all_nw_info$latitude
all_nw_info$Longitude <- all_nw_info$longitude

drops <- c("network_type","number_components", "is_weighted", "cell_values_description", "abundance_description", "latitude", "longitude")
all_nw_info = all_nw_info[ , !(names(all_nw_info) %in% drops)]

In [92]:
colnames(all_nw_info)

[1] "network_name"     "author"           "reference"        "location"        
[5] "location_address" "country"          "Latitude"         "Longitude"

In [93]:
wol_networks_metadata <- left_join(wol_networks_metadata, all_nw_info, by = join_by(network_name == network_name))

In [94]:
rest_networks_metdata =  networks_metadata[networks_metadata$source != "web_of_life", ]

In [104]:
networks_metadata = bind_rows(wol_networks_metadata, rest_networks_metdata)

In [106]:
write.csv(networks_metadata, networks_metadata_path)